# R client example


This notebook serves two purposes:

1. A demo notebook to demonstrate how to use the R client
2. A "smoke-test" noteook to ensure that the methods actually work

For more detailed and verbose tutorials, refer to the [python tutorials](https://hawc.readthedocs.io/api.html#tutorials). The clients should be nearly identical.

## Setup

Install the R client:

```R
devtools::install_github("shapiromatron/hawc", subdir="client/r/rhawc")
```

Or, if in development mode:

```R
# assuming jupyter was started at the root directory
source("../../client/r/rhawc/R/HawcClient.R")
```

Login and authenticate:

In [ ]:
base_url = "https://hawcproject.org"
email = "admin@hawcproject.org"

client = HawcClient(base_url)
client$authenticate(email, getPass::getPass())

HAWC R client version:

In [ ]:
client$version__

---

## Check read-only methods

These are **safe** methods that do not modify any state on HAWC. They pull data from the server.

### Assessment app

In [ ]:
head(client$assessment_public())

In [ ]:
head(client$assessment_bioassay_ml_dataset())

###  Literature app

In [ ]:
assessment_id = 100000044

In [ ]:
head(client$lit_tags(assessment_id))

In [ ]:
head(client$lit_reference_tags(assessment_id))

In [ ]:
head(client$lit_reference_ids(assessment_id))

In [ ]:
ref_ids = client$lit_references(assessment_id)
first_ref_id = ref_ids[1, 'HAWC.ID']
head(ref_ids)
first_ref_id

In [ ]:
head(client$lit_reference(first_ref_id))

### Risk of bias/study evaluation

In [ ]:
assessment_id = 100000053

In [ ]:
head(client$rob_data(assessment_id))

In [ ]:
head(client$rob_full_data(assessment_id))

### Animal bioassay

In [ ]:
assessment_id = 100500065

In [ ]:
head(client$ani_data(assessment_id))

In [ ]:
head(client$ani_data_summary(assessment_id))

In [ ]:
resp = client$ani_endpoints(assessment_id)
length(resp)
resp[[1]]$id

### Epidemiology

In [ ]:
assessment_id = 508

In [ ]:
head(client$epi_data(assessment_id))

In [ ]:
resp = client$epi_endpoints(assessment_id)
length(resp)
resp[[1]]$id

### Epidemiology meta-analysis

In [ ]:
hawc_public_client = HawcClient('https://hawcproject.org')
assessment_id = 94

In [ ]:
head(hawc_public_client$epimeta_data(assessment_id))

### In vitro

In [ ]:
assessment_id = 100000039

In [ ]:
head(client$invitro_data(assessment_id))

### Summary

In [ ]:
assessment_id = 508

In [ ]:
head(client$visual_list(assessment_id))

---

## Checking destructive write methods

These are **unsafe** methods that mutate and modify data on the server. Take care when executing these methods.

In [ ]:
assessment_id = 100500210

In [ ]:
# lit_import_hero
client$lit_import_hero(
    assessment_id = assessment_id,
    title = glue::glue("hero import {round(runif(1, 1, 1000))}"),
    description = "description",
    ids = c(1037843)
)
refs = client$lit_reference_ids(assessment_id)
ref_id = refs[refs[, "hero_id"] == 1037843, "reference_id"]
ref_id

In [ ]:
# reference update
payload = list(year = 2020)
head(client$lit_update_reference(ref_id, data=payload))

In [ ]:
# reference delete
client$lit_delete_reference(ref_id)

### Complete bioassay extraction

In [ ]:
# lit_reference_ids
client$lit_import_hero(
    assessment_id = assessment_id,
    title = glue::glue("hero import {round(runif(1, 1, 1000))}"),
    description = "description",
    ids = c(4322522)
)
refs = client$lit_reference_ids(assessment_id)
ref_id = refs[refs[, "hero_id"] == 4322522, "reference_id"]
ref_id

In [ ]:
# lit_import_reference_tags
tag_id = client$lit_tags(assessment_id)[1, "id"]
csv = glue::glue("reference_id,tag_id\n{ref_id},{tag_id}")
client$lit_import_reference_tags(assessment_id, csv, 'append')

In [ ]:
# study_create
data = list(
    bioassay = TRUE,
    epi = FALSE,
    epi_meta = FALSE,
    in_vitro = FALSE,
    coi_reported = 3,
    coi_details = "",
    funding_source = "Acme industries",
    study_identifier = "4322522",
    contact_author = FALSE,
    ask_author = "",
    published = TRUE,
    summary = "",
    editable = TRUE
)
study = client$study_create(
    reference_id = ref_id,
    short_citation = "York, 2003, 4322522",
    full_citation = "York RG. 2003. Oral (galvage) dosage-range developmental toxicity study of potassium perfluorobutane sulfonate (PFBS) in rats.",
    data = data    
)
head(study)
study_id = study$id

In [ ]:
# rob_create
tryCatch({
    client$rob_create(study_id)
}, error = function(e) {
    cat('err caught (expected)')
})

In [ ]:
# ani_create_experiment
data = list(
    study_id = study_id,     
    name = "30 day oral",     
    type = "St",
    has_multiple_generations = FALSE,
    chemical = "2,3,7,8-Tetrachlorodibenzo-P-dioxin",
    cas = "1746-01-6",
    chemical_source = "ABC Inc.",
    purity_available = TRUE,
    purity_qualifier = "≥",
    purity = 99.9,
    vehicle = "DMSO",
    guideline_compliance = "not reported",
    description = "Details here."
)
experiment = client$ani_create_experiment(data)
experiment_id = experiment$id
experiment_id

In [ ]:
# ani_create_animal_group
data = list(
    experiment_id = experiment_id,     
    name = "Female C57BL/6 Mice",
    species = 2,
    strain = 6,
    sex = "F",
    animal_source = "Charles River",
    lifestage_exposed = "Adult",
    lifestage_assessed = "Adult",
    generation = "",
    comments = "Detailed comments here",
    diet = "...",
    dosing_regime=list(
        route_of_exposure = "OR",
        duration_exposure = 30,
        duration_exposure_text = "30 days",
        duration_observation = 180,
        num_dose_groups = 3,
        positive_control = TRUE,
        negative_control = "VT",
        description = "...",
        doses  = list(
            list(dose_group_id = 0, dose = 0, dose_units_id = 1),
            list(dose_group_id = 1, dose = 50, dose_units_id = 1),
            list(dose_group_id = 2, dose = 100, dose_units_id = 1),
            list(dose_group_id = 0, dose = 0, dose_units_id = 2),
            list(dose_group_id = 1, dose = 3.7, dose_units_id = 2),
            list(dose_group_id = 2, dose = 11.4, dose_units_id = 2)
        )
    )
)
animal_group = client$ani_create_animal_group(data)
animal_group_id = animal_group$id
animal_group_id

In [ ]:
data = list(
    animal_group_id = animal_group_id,
    name = "Relative liver weight",
    system = "Hepatic",
    organ = "Liver",
    effect = "Organ weight",
    effect_subtype = "Relative weight",
    litter_effects = "NA",
    litter_effect_notes = "",
    observation_time = 104,
    observation_time_units = 5,
    observation_time_text = "104 weeks",
    data_location = "Figure 2B",
    expected_adversity_direction = 3,
    response_units = "g/100g BW",
    data_type = "C",
    variance_type = 1,
    confidence_interval = 0.95,
    NOEL = 1,  # should be the corresponding dose_group_id below or -999
    LOEL = 2,  # should be the corresponding dose_group_id below or -999
    FEL = -999,  # should be the corresponding dose_group_id below or -999
    data_reported = TRUE,
    data_extracted = TRUE,
    values_estimated = FALSE,
    monotonicity = 8,
    statistical_test = "ANOVA + Dunnett's test",
    trend_value = 0.0123,
    trend_result = 2,
    diagnostic = "...",
    power_notes = "...",
    results_notes = "...",
    endpoint_notes = "...",
    groups = list(
        list(
            dose_group_id = 0,
            n = 10,
            response = 4.35,
            variance = 0.29,
            significant = FALSE
        ),
        list(
            dose_group_id = 1,
            n = 10,
            response = 5.81,
            variance = 0.47,
            significant = FALSE
        ),
        list(
            dose_group_id = 2,
            n = 10,
            response = 7.72,
            variance = 0.63,
            significant = TRUE,
            significance_level = 0.035
        )
    )
)
endpoint = client$ani_create_endpoint(data)
glue::glue("{base_url}{endpoint$url}")

In [ ]:
# cleanup
client$lit_delete_reference(ref_id)